# <font color='red'> Description du projet </font>

## <font color='blue'>Présentation du problème </font>

L’objectif de ce projet est d’estimer **les temps de réponse et de mobilisation** de la Brigade des Pompiers de Londres. La brigade des pompiers de Londres est le service d'incendie et de sauvetage le plus actif du Royaume-Uni  et l'une des plus grandes organisations de lutte contre l'incendie et de sauvetage au monde.

Le premier jeu de données fourni contient les détails de chaque incident traité depuis janvier 2009. Des informations sont fournies sur la date et le lieu de l'incident ainsi que sur le type d'incident traité. Il est composé de deux fichiers

*   LFB Incident data from 2009 - 2017.xlsx
*   LFB Incident data from 2018 onwards.csv

Le second fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-incident-records/f5066d66-c7a3-415f-9629-026fbda61822/LFB%20Incident%20data%20from%202018%20onwards.csv.xlsx' pour avoir la dernière version du fichier. En effet, les données sont mises à jour tous les mois. Il faut compter au moins 7 minutes pour la lecture des données.

<br>

Le second jeu de données contient les détails de chaque camion de pompiers envoyé sur les lieux d'un incident depuis janvier 2009. Des informations sont fournies sur l'appareil mobilisé, son lieu de déploiement et les heures d'arrivée sur les lieux de l'incident. Il est composé de trois fichiers

*   LFB Mobilisation data from January 2009 - 2014.xlsx
*   LFB Mobilisation data from 2015 - 2020.xlsx
*   LFB Mobilisation data from January 2009 - 2014.xlsx

Le dernier fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-mobilisation-records/3ff29fb5-3935-41b2-89f1-38571059237e/LFB%20Mobilisation%20data%202021%20-%202024.xlsx' pour avoir la dernière version du fichier (mise à jour mensuelle). Il faut compter environ 17 minutes pour la lecture des données.

## <font color='blue'> Etapes précédentes </font>



*   1 - Exploration des données : premières analyses, concaténation des différents fichiers puis jointure des 2 types de données (incident / mobilisation)
*   2 - Data visualisation.ipynb : visualisation des données, étude de la variable à prédire (temps de réponse total) en fonction des variables explicatives, création d'un jeu de données pour la modélisation

Dans le notebook *1 - Exploration des données*, nous avons crée un dataframe df_mobilisation_incident. Il est utilisé ici.

## <font color='blue'>Etapes dans ce notebook </font>

Dans ce notebook, on reprend et affine la dernière section du notebook *2 - Data visualisation.ipynb* : la création des jeux de données pour la modélisation

# <font color='red'>1) Préparation de l'environement de travail </font>

## <font color='blue'>Installation des modules </font>

In [ ]:
#!pip install matplotlib
#!pip install Seaborn
#!pip install openpyxl
#!pip install scipy
#!pip install geopandas
#!pip install scikit-learn
#!pip install statsmodels
#!pip install folium
#!pip install plotly
#!pip install --upgrade seaborn
#!pip install jupyter
#!pip install nbformat
#!pip install fanalysis
#!pip install scientisttools
!pip install plotnine3d


## <font color='blue'>Importation des bibliothèques </font>

In [3]:
import pandas as pd  #Pour les dataframe
import numpy as np #Pour le calcul numérique
import datetime as dt # Pour le calcul sur les dates

# Normalisation pour preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# Libraries divers
from copy import deepcopy  # gestion des copies
from pyproj import Proj # conversion entre les coordonnées British national grid et latitude/longitude
from scipy import stats # notamment pour boxplot

# Pour la séparation du jeu de données
from sklearn.model_selection import train_test_split

# Pour réduction de dimension
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from fanalysis.mca import MCA
#from scientisttools import MCA
%matplotlib inline


## <font color='blue'>Liaison avec le drive (pour travailler sur GoogleColab) </font>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# <font color='red'>2) Récupération des données (cf Exploration de données)</font>

Code pour travailler sur GoogleColab

In [ ]:
dfim = pd.read_csv('/content/gdrive/My Drive/1_Rendu/20241115/LFB incident et mobilisation data.csv', low_memory=False)

# modification du type pour les colonnes date
dfim.DateOfCall=pd.to_datetime(dfim.DateOfCall)
dfim.DateAndTimeMobilised=pd.to_datetime(dfim.DateAndTimeMobilised)
dfim.DateAndTimeMobile=pd.to_datetime(dfim.DateAndTimeMobile)
dfim.DateAndTimeArrived=pd.to_datetime(dfim.DateAndTimeArrived)

# df_2023=dfim[dfim.CalYear==2023].copy(deep=True)

Code pour travail en local

In [ ]:
dfim = pd.read_csv('../Data/Datapreprocessing/LFB incident et mobilisation data.csv',low_memory=False)

# modification du type pour les colonnes date
dfim.DateOfCall=pd.to_datetime(dfim.DateOfCall)
dfim.DateAndTimeMobilised=pd.to_datetime(dfim.DateAndTimeMobilised)
dfim.DateAndTimeMobile=pd.to_datetime(dfim.DateAndTimeMobile)
dfim.DateAndTimeArrived=pd.to_datetime(dfim.DateAndTimeArrived)

# df_2023=dfim[dfim.CalYear==2023]

# <font color='red'> 3) Jeux de données pour la modélisation (avant split) </font>

## <font color='blue'>3.a) Transformation Box-Cox </font>

On a choisi de calculer le lambda de la transformation pour chaque variable sur l'ensemble des données (`dfim`) et non sur la seule année 2023 (`df_2023` précédemment étudié).

On obtient :
- pour `TurnoutTimeSeconds` un lambda de 0,4220
- pour `TravelTimeSeconds` un lambda de 0,5073
- pour `TotalResponseTime` un lambda de 0,4589

In [ ]:
# Liste des variables à transformer
variables = ['TurnoutTimeSeconds', 'TravelTimeSeconds', 'TotalResponseTime']

# Appliquer la transformation de Box-Cox avec lambda_optimal à toutes les variables
for variable in variables:
    dfim[f'boxcox_{variable}'], lambda_optimal = stats.boxcox(dfim[variable])
    print(f"Lambda optimal pour la variable : {variable} est {lambda_optimal}")


In [ ]:
# statistiques descriptives des variables de temps et de leur transformations Box-Cox
# nb :.apply(lambda s: s.apply('{0:.2f}'.format)) permet d'afficher les valeurs arrondis à la décimale plutôt que d'avoir une notation scientifique
dfim[['TurnoutTimeSeconds', 'boxcox_TurnoutTimeSeconds','TravelTimeSeconds', 'boxcox_TravelTimeSeconds', 'TotalResponseTime', 'boxcox_TotalResponseTime']].describe().apply(lambda s: s.apply('{0:.2f}'.format))

## <font color='blue'>3.b) Résumé analyse (notebook 2) </font>

Suite aux analyses effectuées sur les variables explicatives connues *a priori*, nous incluerons dans le modèle
- l'heure de l'incident, `HourOfCall`
- l'arrondissement, `IncGeo_BoroughName` (le quartier, `IncGeo_WardName`,  pourra être étudié dans un second temps)
- la caserne de départ, `DeployedFromStation_Name`
- le type d'incident détaillé, `DetailedIncidentGroup` (après regroupement des catégories *False alarm Malicious* / *Good intent* dans une catégorie *Other*)
- le type de propriété impacté dans l'incident, `PropertyCategory` (le type détaillé `HighPropertyType` pourra être étudié dans un second temps)
- la distance
</br></br>

*Nota Bene* :
</br>
En plus des variables connues *a posteriori* (exactitude de l'adresse, délai, nombre d'appels et nombre de camions appelés sur l'incident), les variables suivantes de `df_2023` n'ont pas fait l'objet d'une analyse
- les dates permettant le calcul du temps de trajet et de réponse : `DateAndTimeMobilised`, `DateAndTimeMobile` et `DateAndTimeArrived`
- les variables ayant permis le calcul de la distance : `Latitude`, `Longitude`, `Easting_rounded`, `Northing_rounded`, `Lat_station` et `Long_station`
- la date exacte de l'incident (`DateOfCall`). Nous avons étudié des variables dérivées comme le jour de la semaine ou le mois et nous avons supposé que l'année n'avait pas d'impact sur le temps de réponse (hypothèse soutenue par nos travaux dans la section 3)
- les variables `IncidentStationGround`, `PumpCount`, `ResourceMobilisationId` et `Resource_Code`. Nous avons un doute sur leur définition et sommes en attente d'un retour à ce sujet (note du 15/11/2024). Si notre compréhension est correct, nous ne pensons pas qu'elles puissent avoir un impact sur le temps de réponse.
- la variable `PropertyType` dont `HighPropertyType` est un regroupement



## <font color='blue'>3.c) Création jeux de données (avant split) </font>

Nous créons 2 jeux de données pour la modélisation.

Dans le premier nous incluons
- les variables explicatives sélectionnées dont `IncGeo_WardName` et `HighPropertyType` mais après la modification de `DetailedIncidentGroup`
- les trois variables de temps sur l'échelle originale et la transformation Box-Cox du temps de réponse total (variable à prédire)
- certaines variables qui nous semble interessant de conserver pour des travaux futurs comme la latitude et la longitude par exemple

Dans le second, nous sélectionnons uniquement la variable à prédire (soit la transformation Box-Cox du temps de réponse total) et les variables explicatives retenues pour la première étape de modélisation (sélection du type de modèles); les variables `IncGeo_WardName` et `HighPropertyType` ne sont donc pas dans ce jeu de données.

### Modification de DetailedIncidentGroup
- on conserve le type *AFA* (alarme incendie automatique)
- on regroupe *False alarm - Good intent* et  *False alarm - Malicious* avec *No action (not false alarm)* en renommant *Other* car dans les 3 cas, il n'y a pas eu d'action à mener
- on regroupe les deux types de *Medical Incident* en une catégorie
- on renomme *Late Call* par *Other Fire* (nom plus "parlant")

Nota Bene : *RTC* signifie *Road Traffic Collision*

In [ ]:
dfim.loc[ (dfim.DetailedIncidentGroup=='False alarm - Good intent') | (dfim.DetailedIncidentGroup=='False alarm - Malicious') | (dfim.DetailedIncidentGroup=='No action (not false alarm)'),'DetailedIncidentGroup']='Other'

dfim.loc[ (dfim.DetailedIncidentGroup=='Medical Incident - Co-responder'),'DetailedIncidentGroup']='Medical Incident'
dfim.loc[ (dfim.DetailedIncidentGroup=='Late Call'),'DetailedIncidentGroup']='Other Fire'


# on vérifie la prise en compte des modifications
dfim.DetailedIncidentGroup.value_counts(normalize=True)

### Catégorisation du temps de réponse

Au cours de notre modélisation, nous souhaitons pouvoir tester des algorithmes de classification. Nous partons de l'hypothèse qu'être capable de prédire un intervalle de temps de réponse pourrait être suffisant. On constate que 98% des temps de réponse sont entre 1 minutes 19 secondes et 13 minutes. Nous avons choisi de créer des catégories de temps correspondant à un intervalle allant de 30 secondes à 6 minutes 30 pour prendre en compte la distribution et avoir au minimum 5% des données dans une catégorie (= un intervalle de temps).

In [ ]:
dfim['TotalResponseTime'].quantile([0.01, 0.025, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.975, 0.99])

Chaque categorie de `ResponseTimeCategory` représente de 4,8 à 11,8% des données. Nous avons choisi les intervalles de temps suivant :
<br> <br>

<table>
<tr>
<th>Categorie</th>
<th>Intervalle</th>
</tr>
<tr>
<td>0</td>
<td> <= 2 minutes 30 secondes </td>
</tr>
<tr>
<td>1</td>
<td>] 2,5 min ; 3 min]</td>
</tr>
<tr>
<td>2</td>
<td>] 3 min; 3,5 min] </td>
</tr>
<tr>
<td>3</td>
<td>] 3,5 min; 4 min ]</td>
</tr>
<tr>
<td>4</td>
<td>] 4 min; 4,5 min]</td>
</tr>
<tr>
<td>5</td>
<td> ] 4,5 min; 5 min ]</td>
</tr>

<tr>
<td>6</td>
<td> ] 5 min; 5,5 min ]</td>
</tr>

<tr>
<td>7</td>
<td> ] 5,5 min; 6 min ]</td>
</tr>

<tr>
<td>8</td>
<td> ] 6 min; 6,5 min ]</td>
</tr>

<tr>
<td>9</td>
<td> ] 6,5 min; 7,5 min ]</td>
</tr>

<tr>
<td>10</td>
<td> ] 7,5 min; 9 min ]</td>
</tr>

<tr>
<td>11</td>
<td> > 9 minutes </td>
</tr>

</table>

In [ ]:
dfim['ResponseTimeCategory']=0
inf=[150, 180, 210, 240, 270, 300, 330, 360, 390, 450, 540]
sup=[180, 210, 240, 270, 300, 330, 360, 390, 450, 540, 630]

j=0
for i in range(0,11):
  j=j+1
  dfim.loc[(dfim.TotalResponseTime>inf[i]) & (dfim.TotalResponseTime<=sup[i]), 'ResponseTimeCategory']=j

dfim.loc[(dfim.TotalResponseTime>=630), 'ResponseTimeCategory']=11


In [ ]:
dfim.ResponseTimeCategory.value_counts(normalize=True)

### Création d'un premier jeu de données "clean"
On supprime les variables
- connues *a posteriori*
- dont la définition n'est pas assez précise
- de dates permettant le calcul du temps de trajet et de réponse

Ce jeu de données a 1 037 713 lignes et 28 colonnes. On renomme certaines variables.

In [ ]:
col=['IncidentNumber', 'TurnoutTimeSeconds', 'TravelTimeSeconds', 'TotalResponseTime', 'boxcox_TotalResponseTime', 'ResponseTimeCategory',
       'DateOfCall', 'CalYear', 'TimeOfCall', 'HourOfCall', 'DayOfWeek', 'Month',
       'IncidentGroup', 'DetailedIncidentGroup',
       'PropertyCategory', 'HighPropertyType',
       'IncGeo_BoroughCode','IncGeo_BoroughName', 'IncGeo_WardCode', 'IncGeo_WardName',
       'DeployedFromStation_Code', 'DeployedFromStation_Name',
       'distance', 'Latitude', 'Longitude', 'IncGeo_Rounded','Lat_station', 'Long_station',
    ]

df_clean=dfim[col].copy(deep=True)

# on renomme certaines colonnes

df_clean.rename(columns={'TurnoutTimeSeconds': 'TurnoutTime',
                             'TravelTimeSeconds' : 'TravelTime',
                             'boxcox_TotalResponseTime' : 'TotalResponseTime_BC',
                             'IncGeo_BoroughCode' : 'BoroughCode',
                             'IncGeo_BoroughName' : 'BoroughName',
                             'IncGeo_WardCode' : 'WardCode',
                             'IncGeo_WardName' : 'WardName',
                             'DeployedFromStation_Code' : 'Station_Code',
                             'DeployedFromStation_Name' : 'Station_Name',
                              'DetailedIncidentGroup' : 'Incident_Type'
                            }, inplace=True)

In [ ]:
df_clean.shape

Enregistrement sur Google Colab

In [ ]:
df_clean.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/CleanDataset.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
df_clean.to_csv('../Data/Datapreprocessing/Complete/CompleteDataset.csv', index=False , encoding='utf-8')

Lecture sur Google Colab

In [ ]:
df_clean = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/CompleteCleanDataset.csv', low_memory=False)

# apres le chargement, il faut modifier le type des dates
df_clean.DateOfCall=pd.to_datetime(df_complete.DateOfCall)

Lecture en local

In [ ]:
df_clean = pd.read_csv('../Data/Datapreprocessing/Complete/CompleteDataset.csv',low_memory=False)

# apres le chargement, il faut modifier le type des dates
df_clean.DateOfCall=pd.to_datetime(df_clean.DateOfCall)

### Création d'un second jeu de données "minimal" pour la modélisation
- On supprime les variables qui ne seront pas utilisées dans la première étape de modélisation (choix du type de modèle) sauf `CalYear` car on va utiliser cette variable pour créer les datasets d'entraînement / validation / test et `IncidentNumber` pour pouvoir faire le lien avec le dataset complet.
- on binarise les variables catégorielles nominales
- on crée 2 variables binaires pour les heures : `H26` qui vaut 1 entre 2 et 6 heures et 0 sinon et `H1117` qui vaut 1 entre 11 et 17 heures et 0 sinon.


*Nota Bene* : à ce stade, on conserve les 2 variables explicatives possibles `TotalResponseTime_BC` et `ResponseTimeCategory`

Ce jeu de données a 1 037 713 lignes et 177 colonnes (après la suppression de `HourOfCall`)

In [ ]:
# selection des colonnes
col=[ 'IncidentNumber', 'TotalResponseTime_BC', 'ResponseTimeCategory', 'CalYear',
       'HourOfCall',
       'Incident_Type', 'PropertyCategory',
       'BoroughCode',
       'Station_Code', 'distance']

df_model=df_clean[col].copy(deep=True)

# création des 2 variables binaires sur l'heure
df_model['H26']=0
df_model.loc[(df_model.HourOfCall>=2) & (df_model.HourOfCall<=6),'H26']=1

df_model['H1117']=0
df_model.loc[(df_model.HourOfCall>=11) & (df_model.HourOfCall<=17),'H1117']=1

# binarisation
df_model=pd.get_dummies(data=df_model, columns=['Incident_Type', 'PropertyCategory', 'BoroughCode','Station_Code'], prefix=['IncTyp', 'PropCat','Borough', 'Station'], prefix_sep='_')


In [ ]:
df_model.shape

In [ ]:
# on supprime la colonne avec les heures
df_model.drop(columns=['HourOfCall'], inplace=True)

df_model.shape

Enregistrement sur Google Colab

In [ ]:
df_model.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/ModelingDataset.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
df_model.to_csv('../Data/Datapreprocessing/Complete/ModelingDataset.csv', index=False , encoding='utf-8')

Lecture sur Google Colab

In [ ]:
df_model = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/ModelingDataset.csv', low_memory=False)



Lecture en local

In [ ]:
df_model = pd.read_csv('../Data/Datapreprocessing/Complete/ModelingDataset.csv', low_memory=False)



# <font color='red'> 4) Split du jeu de données
 </font>







## <font color='blue'>4.a) Split du dataset "complet" </font>

#### Methode pour le split

In [ ]:
def split_dataframe(df, train_ratio, validation_ratio, test_ratio, random_state=None):
    """
    Divise un DataFrame en trois parties : train, validation, et test.

    Parameters:
    - df (pd.DataFrame): Le DataFrame à diviser.
    - train_ratio (float): Proportion des données pour l'ensemble d'entraînement.
    - validation_ratio (float): Proportion des données pour l'ensemble de validation.
    - test_ratio (float): Proportion des données pour l'ensemble de test.
    - random_state (int, optional): Pour reproduire la même division aléatoire.

    Returns:
    - train_df (pd.DataFrame): Jeu d'entraînement.
    - validation_df (pd.DataFrame): Jeu de validation.
    - test_df (pd.DataFrame): Jeu de test.
    """
    total_ratio = train_ratio + validation_ratio + test_ratio
    assert abs(total_ratio - 1.0) < 1e-6, f"Les proportions doivent totaliser 1. Actuellement : {total_ratio}"

    # Mélanger les données
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Étape 1 : Diviser en train + temp (validation + test)
    train_df, temp_df = train_test_split(df, test_size=(1 - train_ratio), random_state=random_state)

    # Étape 2 : Diviser temp en validation et test
    validation_df, test_df = train_test_split(temp_df,
                                              test_size=test_ratio / (test_ratio + validation_ratio),
                                              random_state=random_state)

    return train_df, validation_df, test_df

#### Split du jeu données

Dans un premier temps, on divise le jeu de données `df_model` en 3 jeux de données pour l'entraînement, la validation et le test de modèles. grâce à la fonction `split_dataframe`. Nous utilisons `random_state=2024` pour pouvoir reproduire ce split.

*Nota Bene* : nous avons vérifié et la répartition des incidents par `CalYear` est similaire dans les 3 jeux de données.



In [ ]:
train_df, validation_df, test_df = split_dataframe(df_model, 0.7, 0.15, 0.15, 2024)

# regroupement de train et validation pour la dernière étape de modélisation
train2_df = pd.concat([train_df, validation_df])

## df_model.CalYear.value_counts(normalize=True)
## train_df.CalYear.value_counts(normalize=True)
## validation_df.CalYear.value_counts(normalize=True)
## test_df.CalYear.value_counts(normalize=True)

#### Standardisation de la distance

Une fois le jeu de données divisé, nous standardisons la variable quantitative `distance`. Comme nous avons pu le constater dans le notebook précédent, la répartition de la distance est asymétrique avec une médiane à 1,4 km et un maximum à 40,3 km. Nous avons donc choisi la méthode de standardisation `RobustScaler` qui utilise la médiane et l'interquartile.

Ci-dessous, nous avons appliqué les 3 standardisations à l'ensemble des données de `df_model` pour comparer leurs statistiques descriptives. Cela confirme que la méthode `RobustScaler` est la plus adaptée.

In [ ]:
# statistique descriptive sur la distance
tempo = df_model[['distance']].copy(deep=True)
tempo['dist_rob']=pd.DataFrame(RobustScaler().fit_transform(tempo[['distance']]))
tempo['dist_norm']=pd.DataFrame(StandardScaler().fit_transform(tempo[['distance']]))
tempo['dist_min']=pd.DataFrame(MinMaxScaler().fit_transform(tempo[['distance']]))



display(tempo[['distance', 'dist_norm', 'dist_rob', 'dist_min']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
#del tempo

Nous effectuons 2 standardisations en prenant en compte les données du jeu de données train (`train_df`) et celles des 2 jeux de données regroupées `train_df` et `validation_df` (soit `train2_df`). Chaque standardisation sera utilisée à une étape différente de la modélisation.

In [ ]:
# premier cas : on utiliser la médiane et IQR de train_df
scaler = RobustScaler()
train_df['distStd'] = scaler.fit_transform(train_df[['distance']]) # fit et transform sur train_df
validation_df['distStd'] = scaler.transform(validation_df[['distance']]) # uniquement le transform sur validation_df


# second cas : on utilise la médiane et IQR du regroupement de train_df + validation_df
scaler = RobustScaler()
train2_df['distStd'] = scaler.fit_transform(train2_df[['distance']]) # fit et transform sur train2_df
test_df['distStd'] = scaler.transform(test_df[['distance']]) # uniquement le transfomr sur test_df


In [ ]:
# display(train_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(validation_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train2_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(test_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

In [ ]:
# on conserve uniquement la distance standardisée (on pourra retrouver la distance "originale" dans le dataset df_model et df_clean)
# on supprime la colonne CalYear, on conserve IncidentNumber pour pouvoir retrouver les liens avec df_model et df_clean
train_df.drop(columns=['distance', 'CalYear'], inplace=True)
validation_df.drop(columns=['distance', 'CalYear'], inplace=True)
test_df.drop(columns=['distance', 'CalYear'], inplace=True)
train2_df.drop(columns=['distance', 'CalYear'], inplace=True)

#### Enregistrement des différents jeux de données

Enregistrement sur Google Colab

In [ ]:
train_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset.csv', index=False , encoding='utf-8')
validation_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset.csv', index=False , encoding='utf-8')
train2_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Step3_Dataset.csv', index=False , encoding='utf-8')
test_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
train_df.to_csv('../Data/Datapreprocessing/Complete/Train_Dataset.csv', index=False , encoding='utf-8')
validation_df.to_csv('../Data/Datapreprocessing/Complete/Validation_Dataset.csv', index=False , encoding='utf-8')
train2_df.to_csv('../Data/Datapreprocessing/Complete/Complete/Train_Step3_Dataset.csv', index=False , encoding='utf-8')
test_df.to_csv('../Data/Datapreprocessing/Complete/Complete/Test_Dataset.csv', index=False , encoding='utf-8')

Lecture sur Google Colab

In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset.csv', low_memory=False)
validation_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset.csv', low_memory=False)
train2_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Step3_Dataset.csv', low_memory=False)
test_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset.csv', low_memory=False)



Lecture en local

In [ ]:
train_df = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset.csv', low_memory=False)
validation_df = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset.csv', low_memory=False)
train2_df = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Step3_Dataset.csv', low_memory=False)
test_df = pd.read_csv('../Data/Datapreprocessing/Complete/Test_Dataset.csv', low_memory=False)

## <font color='blue'>4.b) Création de datasets "réduits" </font>

Pour rappel, une fois la binarisation des variables qualitatives effectuée, notre jeu de données avant split contient 1 037 713 lignes et 177 colonnes. Nous nous attendons à des difficultés lors de l'entraînement des modèles (temps d'exécution trop long, problème de convergence pour l'algorithme), surtout pour les plus complexes. Nous créons donc des jeux de données réduits. Pour effectuer la sélection, nous prenons en compte l'année de l'incident. En effet, nous préférons conserver en priorité les données les plus récentes tout en prenant en compte les plus anciennes.

#### Methode pour le split avec réduction

In [ ]:
def split_final(df, ratio_year, ratio_separation, nb_ligne_2024=0, random_state=None):
    """
    Divise un DataFrame par année selon les ratios fournis et retourne 3 DataFrames globaux associé au 3 Series y.

    Parameters:
    - df (pd.DataFrame): Le DataFrame à diviser. Doit contenir la variable CalYear avec des valeurs comprises entre 2014 et 2024
    - ratio_year (list): liste des ratios de valeurs souhaitées dans chaque année
    - ratio_separation (list): liste des ratio train/validation/test
    - random_state (int, optional): Pour reproduire les mêmes divisions aléatoires.

    Returns:
    - train_dfs (pd.DataFrame): Jeu d'entraînement.
    - val_dfs (pd.DataFrame): Jeu de validation.
    - test_dfs (pd.DataFrame): Jeu de test.
    - train_ys (pd.Serie): Résultat du jeu d'entraînement.
    - val_ys (pd.Serie): Résultat du jeu de validation.
    - test_ys (pd.Serie): Résultat du jeu de test.
    """


    # Vérification des ratios
    total_ratio_separation = sum(ratio_separation)
    assert abs(total_ratio_separation - 1.0) < 1e-6, f"Les proportions de séparation doivent totaliser 1. Actuellement : {total_ratio_separation}"
    total_ratio_repartition = sum(ratio_year)
    assert abs(total_ratio_repartition - 1.0) < 1e-6, f"Les proportions de séparation doivent totaliser 1. Actuellement : {total_ratio_repartition}"

    # Filtrer les données pour l'année
    filtered_df = df[df['CalYear'] == 2024]

    ratio_0 = ratio_year[10]
    if (int(nb_ligne_2024*ratio_0)>filtered_df.shape[0]) | (nb_ligne_2024==0):
      nb_ligne = filtered_df.shape[0]
    else:
      nb_ligne = int(nb_ligne_2024*ratio_0)

    # Mélanger les données
    filtered_df = filtered_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    # on selectionne nb_ligne parmi les lignes de filtered_df
    filtered_d = filtered_df[:int(nb_ligne)]

    # Diviser les données en 3 (train, validation et test)
    train_Xs, val_Xs, test_Xs, = split_dataframe(
        filtered_d,
        train_ratio=ratio_separation[0],
        validation_ratio=ratio_separation[1],
        test_ratio=ratio_separation[2],
        random_state=random_state,
    )

    # liste des années en décroissant
    list_year = [2023 , 2022 , 2021 , 2020 , 2019 , 2018 , 2017 , 2016 , 2015 , 2014]

    for indice, year in enumerate(list_year):
        # Récupérer le ratio pour l'année
        ratio = ratio_year[9-indice]

        # Filtrer les données pour l'année
        filtered_df = df[df["CalYear"] == year]

        # Mélanger les données
        filtered_df = filtered_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

        if int(nb_ligne*ratio/filtered_df.shape[0]) > ratio_0:
            print('trop petit ',list_year[indice])
            break
        else :
            # Récupération du nombre de lignes nécessaires
            filtered_d = filtered_df[:int(nb_ligne*ratio/ratio_0)]

        # Diviser les données
        train_X, val_X, test_X = split_dataframe(
            filtered_d,
            train_ratio=ratio_separation[0],
            validation_ratio=ratio_separation[1],
            test_ratio=ratio_separation[2],
            random_state=random_state,
        )

        # Concaténation des données
        train_Xs = pd.concat([train_Xs , train_X] , axis = 0)
        val_Xs = pd.concat([val_Xs , val_X] , axis = 0)
        test_Xs = pd.concat([test_Xs , test_X] , axis = 0)

    return train_Xs, val_Xs, test_Xs


#### Split du jeu de données

On choisit de faire une première réduction de sorte à faire une sélection sur les années 2014 à 2023 et à avoir la totalité des lignes 2024 (les plus récentes), soit les 92087 incidents.

In [ ]:
# premiere reduction avec la totalité des données sur 2024 (92 087 lignes)
# au total 306 953 lignes (92 087/0.3)
train_reduit1, val_reduit1, test_reduit1 = split_final(
    df_model,
    ratio_separation = [0.7 , 0.15 , 0.15],
    ratio_year = [0.02 , 0.02 , 0.02 , 0.02, 0.02 , 0.05 , 0.05,  0.1 , 0.1 , 0.3, 0.3 ],
    nb_ligne_2024=df_model.shape[0],
    random_state = 2024)

In [ ]:
print('Nb d\'incidents en 2024 :', df_model[df_model.CalYear==2024].shape[0])
print('Nb d\'incidents sur 10 ans avec 30% en 2024 :', np.round(df_model[df_model.CalYear==2024].shape[0]/0.3,0))
print('Nb d\'incidents sélectionnés (vérification) : ', train_reduit1.shape[0] +  test_reduit1.shape[0] + val_reduit1.shape[0])
print('dont 2024 : ', train_reduit1[train_reduit1.CalYear==2024].shape[0] +  test_reduit1[test_reduit1.CalYear==2024].shape[0] + val_reduit1[val_reduit1.CalYear==2024].shape[0])

In [ ]:
print('Répartition des incidents par année (vérification sur train) :\n')
display(train_reduit1.CalYear.value_counts(normalize=True))

Nous avons fait une seconde réduction de la taille du jeu de données de sorte à limiter la taille à 200 000 incidents. Pour maximiser la prise en compte des incidents 2024, nous avons modifié les pourcentages.

In [ ]:
train_reduit2, val_reduit2, test_reduit2 = split_final(
    df_model,
    ratio_separation = [0.7 , 0.15 , 0.15],
    ratio_year = [0.02 , 0.02 , 0.02 , 0.02, 0.02 , 0.05 , 0.05,  0.075 , 0.075 , 0.25, 0.4],
    nb_ligne_2024=200000,
    random_state = 2024)

In [ ]:
print('Nb d\'incidents en 2024 :', df_model[df_model.CalYear==2024].shape[0])
print('Nb d\'incidents sélectionnés (vérification) : ', train_reduit2.shape[0] +  test_reduit2.shape[0] + val_reduit2.shape[0])
print('dont 2024 : ', train_reduit2[train_reduit2.CalYear==2024].shape[0] +  test_reduit2[test_reduit2.CalYear==2024].shape[0] + val_reduit2[val_reduit2.CalYear==2024].shape[0])

In [ ]:
print('Répartition des incidents par année (vérification sur train) :\n')
display(train_reduit2.CalYear.value_counts(normalize=True))

#### Standardisation de la distance

In [ ]:
# regroupement de train et validation pour la dernière étape de modélisation
train2_reduit1 = pd.concat([train_reduit1, val_reduit1])
train2_reduit2 = pd.concat([train_reduit2, val_reduit2])

In [ ]:
# premier cas : on utiliser la médiane et IQR de train_df
scaler = RobustScaler()
train_reduit1['distStd'] = scaler.fit_transform(train_reduit1[['distance']]) # fit et transform sur train_df
val_reduit1['distStd'] = scaler.transform(val_reduit1[['distance']]) # uniquement le transfomr sur validation_df


# second cas : on utilise la médiane et IQR du regroupement de train_df + validation_df
scaler = RobustScaler()
train2_reduit1['distStd'] = scaler.fit_transform(train2_reduit1[['distance']]) # fit et transform sur train2_df
test_reduit1['distStd'] = scaler.transform(test_reduit1[['distance']]) # uniquement le transfomr sur test_df

In [ ]:
# premier cas : on utiliser la médiane et IQR de train_df
scaler = RobustScaler()
train_reduit2['distStd'] = scaler.fit_transform(train_reduit2[['distance']]) # fit et transform sur train_df
val_reduit2['distStd'] = scaler.transform(val_reduit2[['distance']]) # uniquement le transfomr sur validation_df


# second cas : on utilise la médiane et IQR du regroupement de train_df + validation_df
scaler = RobustScaler()
train2_reduit2['distStd'] = scaler.fit_transform(train2_reduit2[['distance']]) # fit et transform sur train2_df
test_reduit2['distStd'] = scaler.transform(test_reduit2[['distance']]) # uniquement le transfomr sur test_df

In [ ]:
# display(train_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(val_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train2_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(test_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(val_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train2_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(test_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

In [ ]:
# on conserve uniquement la distance standardisée (on pourra retrouver la distance "originale" dans le dataset df_model et df_clean)
# on supprime la colonne CalYear, on conserve IncidentNumber pour pouvoir retrouver les liens avec df_model et df_clean
train_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)
val_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)
test_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)
train2_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)

train_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)
val_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)
test_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)
train2_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)

#### Enregistrement des différents jeux de données

Enregistrement sur Google Colab

In [ ]:
train_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Dataset.csv', index=False , encoding='utf-8')
val_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Validation_Dataset.csv', index=False , encoding='utf-8')
train2_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Step3_Dataset.csv', index=False , encoding='utf-8')
test_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Test_Dataset.csv', index=False , encoding='utf-8')

train_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Dataset.csv', index=False , encoding='utf-8')
val_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Validation_Dataset.csv', index=False , encoding='utf-8')
train2_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Step3_Dataset.csv', index=False , encoding='utf-8')
test_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Test_Dataset.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
train_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Train_Dataset.csv', index=False , encoding='utf-8')
val_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Validation_Dataset.csv', index=False , encoding='utf-8')
train2_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Train_Step3_Dataset.csv', index=False , encoding='utf-8')
test_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Test_Dataset.csv', index=False , encoding='utf-8')

train_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Train_Dataset.csv', index=False , encoding='utf-8')
val_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Validation_Dataset.csv', index=False , encoding='utf-8')
train2_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Train_Step3_Dataset.csv', index=False , encoding='utf-8')
test_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Test_Dataset.csv', index=False , encoding='utf-8')

Lecture depuis Google Colab

In [ ]:
train_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Dataset.csv', low_memory=False)
val_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Validation_Dataset.csv', low_memory=False)
train2_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Step3_Dataset.csv', low_memory=False)
test_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Test_Dataset.csv', low_memory=False)

train_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Dataset.csv', low_memory=False)
val_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Validation_Dataset.csv', low_memory=False)
train2_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Step3_Dataset.csv', low_memory=False)
test_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Test_Dataset.csv', low_memory=False)

Lecture depuis un enregistrement en local

In [ ]:
train_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Train_Dataset.csv', low_memory=False)
val_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Validation_Dataset.csv', low_memory=False)
train2_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Train_Step3_Dataset.csv', low_memory=False)
test_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Test_Dataset.csv', low_memory=False)

train_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Train_Dataset.csv', low_memory=False)
val_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Validation_Dataset.csv', low_memory=False)
train2_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Train_Step3_Dataset.csv', low_memory=False)
test_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Test_Dataset.csv', low_memory=False)

## <font color='blue'>4.c) Réduction de dimension </font>

Sur google colab

In [ ]:
df_complete = pd.read_csv('/content/gdrive/My Drive/1_Rendu/20241115/CleanDataset.csv', low_memory=False)

En local

In [14]:
df_complete = pd.read_csv('../Data/Datapreprocessing/Complete/CompleteDataset.csv', low_memory=False)

Recupération de 'TotalResponseTimeCategory'

In [15]:
#Catégorisation du temps de réponse en catégorielle
df_complete['ResponseTimeCategory']=0
inf=[150, 180, 210, 240, 270, 300, 330, 360, 390, 450, 540]
sup=[180, 210, 240, 270, 300, 330, 360, 390, 450, 540, 630]

j=0
for i in range(0,11):
  j=j+1
  df_complete.loc[(df_complete.TotalResponseTime>inf[i]) & (df_complete.TotalResponseTime<=sup[i]), 'ResponseTimeCategory']=j

df_complete.loc[(df_complete.TotalResponseTime>=630), 'ResponseTimeCategory']=11

del inf,sup,j,i

In [16]:
df_complete['HCat']='H'
df_complete.loc[(df_complete.HourOfCall>=2) & (df_complete.HourOfCall<=6),'HCat']='H26'
df_complete.loc[(df_complete.HourOfCall>=11) & (df_complete.HourOfCall<=17),'HCat']='H1117'

Pour notre étude la variable cible 'TotalResponseTime_BC' en continue ou 'ResponseTimeCategory' en discrèt.

Pour la prédiction nous garderons 'HCat','DetailedIncidentGroup', 'PropertyCategory','BoroughCode','Station_Code' qui sont des variables qualitatives possédant respectivement  3 modalités, 27 modalités, 9 modalités, 33 modalités, 102 modalités, et 'distance' qui est une variable quantitative.

Lors de la binarisation des données (pour faire fonctionner des algorithme de prédiction) nous avons donc un dataset de 3+27+9+33+102 = 174 variables. Ce nombres important de variables est génant car il va augmenter fortement le temps de calcul des différents algorithme. 

### Mise en place de ACM sur le jeu de données complet.

**Présentation** une Analyse en Correspondance Multiple (ACM) est une méthode statistique qui permet d'explorer et de visualiser des données qualitatives.

**Objectifs :**
- Découvrir des liens cachés : Elle montre si certaines réponses à des questions sont souvent associées ensemble.
- Réduire la complexité : Elle transforme un tableau complexe en un graphique plus simple, où les points représentent les personnes et les groupes de réponses similaires sont proches les uns des autres.
- Visualiser les données: Tu permet de voir d'un coup d'œil les grandes tendances dans les données.

L'ACM est accéssible directement en Python à l'aide du package fanalysis (lien pour explication : https://github.com/OlivierGarciaDev/fanalysis/blob/master/doc/mca_tutorial.ipynb).

**Principe :** ACP (quantitative) -> AFC (2 qualitatives) -> ACM (plusieurs qualitatives)
ACP sur tableau de contingence avec une métrique discrète du khi²

#### Approche visualisation

Dans une approche "datamining", l'ACM vise à décrire un jeu de données

Cette première étape peu être faite dans la datavisualisation (a savoir notebook 2)

Notre DataFrame contient, à l'heure actuelle, les colonnes suivantes :
- 'TotalResponseTime_BC' ou 'ResponseTimeCategory' la variable cible -> considérée comme variable suplémentaire.
- 'HCat', 'DetailedIncidentGroup', 'PropertyCategory', 'BoroughCode', 'Station_Code' nos variables descriptives (catégorie d'heure, détail d'incident, type de propriété, code arrondissement et code station).
- 'distance' notre variable quantitative que nous traiterons en variable suplémentaire.

In [18]:
# Sélection des variables actives
data_active = df_complete[['HCat', 'PropertyCategory','BoroughCode','Station_Code']]


In [19]:
# Création de l'objet MCA et ajustement du modèle
mca = MCA() 

In [20]:
# Application de la MCA à data_active

# Création d'une matrice (tableau de tableau) contenant la description de chaque individus
X = data_active.values

# Entrainement de la MCA sur X
mca.fit(X)

del X
del data_active

**Analyse des valeurs propres**

L'attribut my_mca.eig_ contient :

- en 1ère ligne : les valeurs propres en valeur absolue
- en 2ème ligne : les valeurs propres en pourcentage de la variance totale
- en 3ème ligne : les valeurs propres en pourcentage cumulé de la variance totale

In [21]:
# Affichage des 10 premières valeurs propres en pourcentage 
mca.eig_[1,:10]

array([1.42426287, 1.39180018, 1.38992403, 1.37492961, 1.36902312,
       1.35556591, 1.34453241, 1.33962985, 1.32969187, 1.31844932])

Ce tableau des valeurs valeurs propres nous indique que :
- Le première axe contidendra 13% des informations contenues dans le jeu de données
- Le deuxième axe contiendra 12% des informations contenues dans le jeu de données 
- et ainsi de suite

Pour choisir le nombre d'axe à garder, on regarde l'inertie totale expliquée (somme des valeurs propres). Pour notre jeux de données on souhaite garder 95% de l'information.

In [22]:
arg_max =np.argmax(mca.eig_[2,:]>=95)
print(arg_max)

109


Pour conserver 95% des informations totales il faut conserver 133 dimensions (correspondant à 133 colonnes : col_active + 'distance') soit 76%. Ce qui conrrespond à une diminution de 24% de la taille des données.

Création d'une nouvelle ACM conservant 95% des informations

In [23]:
# Création de la nouvelle ACM
mca = MCA(arg_max+1,stats=False)

del arg_max

In [24]:
# Application de de la MCA à df_train
mca.fit(df_complete[['HCat', 'PropertyCategory','BoroughCode','Station_Code']].values)

MCA(n_components=np.int64(110), stats=False)

In [47]:
X1 = mca.transform(df_complete.loc[:50000,['HCat', 'PropertyCategory','BoroughCode','Station_Code']].values)
X1


array([[ 0.93191863, -1.86715895,  1.54488929, ..., -0.02078472,
        -0.00450026, -0.00364009],
       [ 0.93191863, -1.86715895,  1.54488929, ..., -0.02078472,
        -0.00450026, -0.00364009],
       [ 0.93806702, -1.85681744,  1.53574448, ..., -0.02098635,
        -0.00504444, -0.00370071],
       ...,
       [ 0.34167795, -1.31322527,  0.93865301, ..., -0.01715964,
         0.00482759, -0.00384971],
       [ 0.34167795, -1.31322527,  0.93865301, ..., -0.01715964,
         0.00482759, -0.00384971],
       [ 0.33001641, -1.29928115,  0.93497946, ..., -0.01773919,
         0.00529323, -0.00408483]])

In [48]:
X1.shape

(50001, 143)

In [51]:
def transfo_temp(mca,X):
    '''
    Transformation grace à la MCA du dataFrame par jeu de 50 000 lignes

    Args :
        mca -> transformateur de MCA
        X dataFrame
    '''
    df_mca = pd.DataFrame()
    for i in range(0,(X.shape[0]//50000 + 1)):

        print(i)
    
        # Application de de la MCA à df
        X = mca.transform(X.iloc[i*50000+1:(i+1)*50000,:].values)[:,:mca.n_components_]

        # Transformation en dataFrame
        X = pd.DataFrame(X)

        # Concatenation
        df_mca = pd.concat([df_mca,X],axis=0)

    return df_mca

In [85]:
(168-131)/168*100

22.023809523809522

In [56]:
X_mca = transfo_temp(mca,df_complete[['HCat', 'PropertyCategory','BoroughCode','Station_Code']])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [54]:
Y.head()

,0,1,2,3,4,5,6,7,8,9,...,133,134,135,136,137,138,139,140,141,142
0,0.931919,-1.867159,1.544889,0.327772,-0.686327,0.709595,-1.024131,0.696349,-0.126618,-1.630982,...,-0.044393,0.052782,0.012724,0.021583,-0.033837,-0.006488,-0.005117,-0.020785,-0.004500,-0.003640
1,0.938067,-1.856817,1.535744,0.298214,-0.713308,0.715453,-0.978410,0.716468,-0.136345,-1.640478,...,-0.047070,0.053224,0.012258,0.023235,-0.032338,-0.006752,-0.003878,-0.020986,-0.005044,-0.003701
2,0.931919,-1.867159,1.544889,0.327772,-0.686327,0.709595,-1.024131,0.696349,-0.126618,-1.630982,...,-0.044393,0.052782,0.012724,0.021583,-0.033837,-0.006488,-0.005117,-0.020785,-0.004500,-0.003640
3,0.949729,-1.870762,1.539418,0.294337,-0.706819,0.713961,-0.973636,0.707017,-0.130866,-1.647150,...,-0.047175,0.052872,0.012449,0.023755,-0.032867,-0.006228,-0.004513,-0.020407,-0.005510,-0.003466
4,0.943580,-1.881103,1.548563,0.323896,-0.679838,0.708103,-1.019357,0.686898,-0.121139,-1.637654,...,-0.044498,0.052431,0.012916,0.022104,-0.034365,-0.005965,-0.005752,-0.020205,-0.004966,-0.003405


In [63]:
def ajout_col(X_mca,X):
    X_mca = pd.DataFrame(np.hstack([X_mca.value, X.loc[:,['distStd']].values]))
    X_mca = pd.DataFrame(np.hstack([X.loc[:,['TotalResponseTime_BC']].values,X_mca.value]))
    X_mca = pd.DataFrame(np.hstack([X.loc[:,['ResponseTimeCategory']].values,X_mca.value]))
    X_mca = X_mca.rename(columns={X_mca.columns[0]: 'TotalResponseTime_BC'})
    X_mca = X_mca.rename(columns={X_mca.columns[1]: 'ResponseTimeCategory'})
    X_mca = X_mca.rename(columns={X_mca.columns[-1]: 'distStd'})
    return X_mca


In [80]:
def ajout_col2(X_mca,X):
    # Ajouter la colonne 'distStd'
    X_mca['distance'] = X['distance']

    # Ajouter la colonne 'TotalResponseTime_BC'
    X_mca.insert(0, 'TotalResponseTime_BC', X['TotalResponseTime_BC']) 

    # Ajouter la colonne 'ResponseTimeCategory'
    X_mca.insert(1, 'ResponseTimeCategory', X['ResponseTimeCategory'])

    return X_mca

In [77]:
X2 = X_mca.copy()
X2['distance']=df_complete['distance']

In [81]:
X = ajout_col2(X_mca,df_complete)

In [82]:
X.head()

,TotalResponseTime_BC,ResponseTimeCategory,0,1,2,3,4,5,6,7,...,134,135,136,137,138,139,140,141,142,distance
0,2.381590,0,0.931919,-1.867159,1.544889,0.327772,-0.686327,0.709595,-1.024131,0.696349,...,0.052782,0.012724,0.021583,-0.033837,-0.006488,-0.005117,-0.020785,-0.004500,-0.003640,3770.970
1,40.327167,11,0.938067,-1.856817,1.535744,0.298214,-0.713308,0.715453,-0.978410,0.716468,...,0.053224,0.012258,0.023235,-0.032338,-0.006752,-0.003878,-0.020986,-0.005044,-0.003701,3781.096
2,34.787931,10,0.931919,-1.867159,1.544889,0.327772,-0.686327,0.709595,-1.024131,0.696349,...,0.052782,0.012724,0.021583,-0.033837,-0.006488,-0.005117,-0.020785,-0.004500,-0.003640,3785.636
3,33.631018,9,0.949729,-1.870762,1.539418,0.294337,-0.706819,0.713961,-0.973636,0.707017,...,0.052872,0.012449,0.023755,-0.032867,-0.006228,-0.004513,-0.020407,-0.005510,-0.003466,3802.989
4,34.287106,10,0.943580,-1.881103,1.548563,0.323896,-0.679838,0.708103,-1.019357,0.686898,...,0.052431,0.012916,0.022104,-0.034365,-0.005965,-0.005752,-0.020205,-0.004966,-0.003405,3757.621


#### Mise en place de l'ACM sur le jeu d'entrainement, de validation et de test

In [7]:
def transfo_mca(col_desc_quali,df_train,df_validation=None,df_test=None):
    """
    Appliquer une réduction de dimension par ACM en conservant 95% des informations

    Args:
        col_desc_quali -> list : nom des variables qualitatives
        df_train, df_validation, df_test -> dataFrame d'entrainement, de validation et de test. 
    """

    data_desc = df_train[col_desc_quali]

    # Création de l'objet MCA et ajustement du modèle
    mca = MCA(row_labels=data_desc.index.values, var_labels=data_desc.columns.values) 

    # Création d'une matrice (tableau de tableau) contenant la description de chaque individus
    X = data_desc.values

    # Entrainement de la MCA sur X
    mca.fit(X)

    # Récupération de l'indice pour 95% des informations
    arg_max =np.argmax(mca.eig_[2,:]>=95)

    # Création de la nouvelle ACM
    mca = MCA(n_components=arg_max+1,row_labels=data_desc.index.values, var_labels=data_desc.columns.values)

    # Application de de la MCA à df_train
    df_train_mca = mca.fit_transform(df_train[col_desc_quali].values)

    # Transformation en dataFrame
    df_train_mca = pd.DataFrame(df_train_mca)


    # Ajout de la variable 'distance'
    df_train_transfo = pd.concat([df_train_mca,df_train['distance']],axis=1) 

    # Application de de la MCA à df_validation 
    df_validation_mca = mca.transform(df_validation[col_desc_quali].values)

    # Transformation en dataFrame
    df_validation_mca = pd.DataFrame(df_validation_mca)

    # Ajout de la variable 'distance'
    df_validation_transfo = pd.concat([df_validation_mca,df_validation['distance']],axis=1) 

    # Application de de la MCA à df_test
    df_test_mca = mca.transform(df_test[col_desc_quali].values)

    # Transformation en dataFrame
    df_test_mca = pd.DataFrame(df_test_mca)

    # Ajout de la variable 'distance'
    df_test_transfo = pd.concat([df_test_mca,df_train['distance']],axis=1) 
    
    return df_train_transfo,df_validation_transfo,df_test_transfo

    

In [49]:
def transfo_mca(col_desc_quali,df_train):
    """
    Appliquer une réduction de dimension par ACM en conservant 95% des informations

    Args:
        col_desc_quali -> list : nom des variables qualitatives
        df_train, df_validation, df_test -> dataFrame d'entrainement, de validation et de test. 
    """

    data_desc = df_train[col_desc_quali]

    # Création de l'objet MCA et ajustement du modèle
    mca = MCA(row_labels=data_desc.index.values, var_labels=data_desc.columns.values) 

    # Création d'une matrice (tableau de tableau) contenant la description de chaque individus
    X = data_desc.values

    # Entrainement de la MCA sur X
    mca.fit(X)

    # Récupération de l'indice pour 95% des informations
    arg_max =np.argmax(mca.eig_[2,:]>=95)

    # Création de la nouvelle ACM
    mca = MCA(n_components=arg_max+1,row_labels=data_desc.index.values, var_labels=data_desc.columns.values)

    # Entrainement du modèle
    mca.fit(df_train[col_desc_quali].values)

    # Application de de la MCA à df_train
    X = mca.transform(df_train.loc[:50000,col_desc_quali].values)[:,:arg_max+1]

    # Transformation en dataFrame
    df_train_transfo = pd.DataFrame(X)

    for i in range(1,(df_train.shape[0]//50000 + 1)):
    

        # Application de de la MCA à df_train
        X = mca.transform(df_train.loc[i*50000+1:(i+1)*50000,col_desc_quali].values)[:,:arg_max+1]

        # Transformation en dataFrame
        X = pd.DataFrame(X)

        # Concatenation
        df_train_transfo = pd.concat([df_train_transfo,X],axis=0)

    # Ajout de la variable 'distance'
    # Concaténation en utilisant numpy.hstack
    df_train_transfo = pd.DataFrame(np.hstack([df_train_transfo.values, df_train.loc[:,['distance']].values]))
    df_train_transfo = df_train_transfo.rename(columns={df_train_transfo.columns[-1]: 'distance'})

    return df_train_transfo

In [50]:
df_train_transfo=transfo_mca(['HCat','DetailedIncidentGroup', 'PropertyCategory','BoroughCode','Station_Code'],
                                                                   df_complete.iloc[:100000,:])



train (100000, 63)
Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       49989, 49990, 49991, 49992, 49993, 49994, 49995, 49996, 49997, 49998],
      dtype='int64', length=100000)
base (100000, 1)
RangeIndex(start=0, stop=100000, step=1)


In [51]:
df_train_transfo.columns

Index([         0,          1,          2,          3,          4,          5,
                6,          7,          8,          9,         10,         11,
               12,         13,         14,         15,         16,         17,
               18,         19,         20,         21,         22,         23,
               24,         25,         26,         27,         28,         29,
               30,         31,         32,         33,         34,         35,
               36,         37,         38,         39,         40,         41,
               42,         43,         44,         45,         46,         47,
               48,         49,         50,         51,         52,         53,
               54,         55,         56,         57,         58,         59,
               60,         61,         62, 'distance'],
      dtype='object')

In [8]:
def entrainement_model(X):
    '''
    Construction d'une MCA pour garder 95% de l'information du dataFrame

    Arg : X -> dataFrame
    '''
    # Création de l'objet MCA et ajustement du modèle
    mca = MCA() 

    # Création d'une matrice (tableau de tableau) contenant la description de chaque individus
    X = X.values

    # Entrainement de la MCA sur X
    mca.fit(X)

    # Récupération de l'indice pour 95% des informations
    arg_max =np.argmax(mca.eig_[2,:]>=95)

    # Création de la nouvelle ACM
    mca = MCA(n_components=arg_max+1,stats=False)

    # Entrainement du modèle
    mca.fit(X)

    return mca

def transfo_temp(mca,X):
    '''
    Transformation grace à la MCA du dataFrame par jeu de 50 000 lignes

    Args :
        mca -> transformateur de MCA
        X dataFrame
    '''
    df_mca = pd.DataFrame()
    for i in range(0,(X.shape[0]//50000 + 1)):
    
        # Application de de la MCA à df
        X = mca.transform(X.iloc[i*50000+1:(i+1)*50000,:].values)[:,:mca.n_components_]

        # Transformation en dataFrame
        X = pd.DataFrame(X)

        # Concatenation
        df_mca = pd.concat([df_mca,X],axis=0)

    return df_mca

def transfo_mca(col_desc_quali,X_train,X_val,X_test):
    '''
    Appliquer une réduction de dimension par ACM en conservant 95% des informations

    Args:
        col_desc_quali -> list : nom des variables qualitatives
        df_train, df_validation, df_test -> dataFrame d'entrainement, de validation et de test. 
    '''
    # Entrainement du modèle
    mca = entrainement_model(X_train.loc[:,col_desc_quali])

    # Transformation de X_train
    X_train_mca = transfo_temp(mca,X_train.loc[:,col_desc_quali])
    X_train_mca = pd.DataFrame(np.hstack([X_train.values, X_train.loc[:,['distance']].values]))
    X_train_mca = X_train_mca.rename(columns={X_train_mca.columns[-1]: 'distance'})
    

    # Transformation de X_validation
    X_val_mca = transfo_temp(mca,X_val.loc[:,col_desc_quali])
    X_val_mca = pd.DataFrame(np.hstack([X_val.values, X_val.loc[:,['distance']].values]))
    X_val_mca = X_val_mca.rename(columns={X_val_mca.columns[-1]: 'distance'})

    # Transformation de X_train
    X_test_mca = transfo_temp(mca,X_test.loc[:,col_desc_quali])
    X_test_mca = pd.DataFrame(np.hstack([X_train.values, X_test.loc[:,['distance']].values]))
    X_test_mca = X_test_mca.rename(columns={X_test_mca.columns[-1]: 'distance'})

    return X_train_mca,X_val_mca,X_test_mca
    

In [10]:
df_complete.iloc[:100000,:].columns

Index(['IncidentNumber', 'TurnoutTime', 'TravelTime', 'TotalResponseTime',
       'TotalResponseTime_BC', 'ResponseTimeCategory', 'DateOfCall', 'CalYear',
       'TimeOfCall', 'HourOfCall', 'DayOfWeek', 'Month', 'IncidentGroup',
       'Incident_Type', 'PropertyCategory', 'HighPropertyType', 'BoroughCode',
       'BoroughName', 'WardCode', 'WardName', 'Station_Code', 'Station_Name',
       'distance', 'Latitude', 'Longitude', 'IncGeo_Rounded', 'Lat_station',
       'Long_station', 'HCat'],
      dtype='object')

In [11]:
df1,df2,df3=transfo_mca(['HCat','PropertyCategory','BoroughCode','Station_Code'],
                                                                   df_complete.iloc[:100000,:],
                                                                   df_complete.iloc[:100000,:],
                                                                   df_complete.iloc[:100000,:])

In [12]:
df1.shape

(100000, 30)

In [49]:
test = df_complete.loc[1000000:1050000,['HCat','DetailedIncidentGroup', 'PropertyCategory','BoroughCode','Station_Code']]